In [1]:
import cv2
import numpy as np
import mediapipe as mp
from ultralytics import YOLO
import random
from sklearn.cluster import KMeans
from skimage.feature import graycomatrix, graycoprops

# Function to get dominant color
def get_dominant_color(image, k=3):
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(pixels)
    dominant_color = kmeans.cluster_centers_[np.argmax(np.bincount(kmeans.labels_))]
    return tuple(map(int, dominant_color))

# Function to get texture features using GLCM
def get_texture_features(gray_image):
    glcm = graycomatrix(gray_image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    return contrast, energy, homogeneity

# Initialize YOLOv8 for person detection
model = YOLO("yolov8n.pt")

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Load Image
image_path = "friendsfinale.webp"  # Change to your image path
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Detect humans using YOLO
results = model(image)

# Extract person bounding boxes
bboxes = []
for r in results:
    for box in r.boxes:
        cls = int(box.cls[0])  # Get class index
        if cls == 0:  # Class 0 is "person" in COCO dataset
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box
            bboxes.append((x1, y1, x2, y2))

# Initialize Pose Estimation
pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, enable_segmentation=False)

# Generate random colors for each person
person_colors = [(random.randint(50, 255), random.randint(50, 255), random.randint(50, 255)) for _ in range(len(bboxes))]

# Process Each Person in Image
for i, (x1, y1, x2, y2) in enumerate(bboxes):
    # Crop Person Region
    person_crop = image_rgb[y1:y2, x1:x2]
    
    # Display each cropped person
    cv2.imshow("Person Crop", person_crop)
    cv2.waitKey(0)
    
    # Get dominant color
    dominant_color = get_dominant_color(person_crop)
    print(f"Person {i + 1} Dominant Color: {dominant_color}")
    
    # Convert to grayscale for texture analysis
    gray_crop = cv2.cvtColor(person_crop, cv2.COLOR_RGB2GRAY)
    contrast, energy, homogeneity = get_texture_features(gray_crop)
    print(f"Person {i + 1} Texture - Contrast: {contrast}, Energy: {energy}, Homogeneity: {homogeneity}")
    
    # Process Pose
    results = pose.process(person_crop)
    
    # Draw Pose Landmarks and Connections
    if results.pose_landmarks:
        h, w, _ = person_crop.shape
        landmarks = []

        for landmark in results.pose_landmarks.landmark:
            cx, cy = int(landmark.x * w) + x1, int(landmark.y * h) + y1
            landmarks.append((cx, cy))
            cv2.circle(image, (cx, cy), 5, person_colors[i], -1)

        # Draw lines connecting the keypoints
        for connection in mp_pose.POSE_CONNECTIONS:
            start_idx, end_idx = connection
            if start_idx < len(landmarks) and end_idx < len(landmarks):
                cv2.line(image, landmarks[start_idx], landmarks[end_idx], person_colors[i], 3)

# Display the result
cv2.imshow("Multi-Person Pose Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 384x640 6 persons, 42.4ms
Speed: 1.9ms preprocess, 42.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Person 1 Dominant Color: (116, 78, 38)
Person 1 Texture - Contrast: 365.9497829861111, Energy: 0.03818534357324686, Homogeneity: 0.3038077753421453
Person 2 Dominant Color: (36, 33, 23)
Person 2 Texture - Contrast: 277.5979139504563, Energy: 0.028976383248322565, Homogeneity: 0.2603097167603211
Person 3 Dominant Color: (45, 34, 21)
Person 3 Texture - Contrast: 553.5677019270191, Energy: 0.014528322592768235, Homogeneity: 0.1497130040640073
Person 4 Dominant Color: (38, 34, 24)
Person 4 Texture - Contrast: 379.3030484957642, Energy: 0.0205878505441849, Homogeneity: 0.1789559703013084
Person 5 Dominant Color: (87, 31, 22)
Person 5 Texture - Contrast: 269.2828611707841, Energy: 0.0238858059302232, Homogeneity: 0.26521442357321456
Person 6 Dominant Color: (28, 18, 14)
Person 6 Texture - Contrast: 241.7338778844191, Energy: 0.05185322140372436, Homogeneity: 0.287733